####Instalações necessarias

In [0]:
print ("Olá, Mundo!")

In [0]:
%pip install xlrd>=2.0.1
%pip install openpyxl
%restart_python


####Bibliotecas utlizadas

In [0]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns, sklearn as sklearn, statistics as st
from sklearn.model_selection import train_test_split as tts
from sklearn.linear_model import LogisticRegression as lr
from sklearn.linear_model import LinearRegression as lm
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.ensemble import RandomForestRegressor as rfr
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import accuracy_score as acs
from sklearn.metrics import precision_score as ps
from sklearn.metrics import recall_score as rs
from sklearn.metrics import f1_score as fs
from sklearn.metrics import confusion_matrix as cm
from sklearn.metrics import classification_report as cr

####Analise Inicial dos dados fornecidos:

In [0]:
import pandas as pd

#df = data frame
# variavel "dados" = local do arquivo xlsx
dados = "/Workspace/Users/ahmad.jmazloum@gmail.com/Data__1_.xlsx"

df = pd.read_excel(dados, sheet_name="Data")
print ("Formato (linhas, colunas):", df.shape)
#dtypes = df.dtypes (função para ver o tipo de dados )
print("\nTipos de dados:")
print(df.dtypes)
# Visualizando os primeiros registros
df.head()

##### Tratando as variaveis ↓

Código utilizado ↓

df[] = df.[].replace({}) → Utilizado para modificação numérica dos dados, podendo alterar as informações para melhor analise e tratamento dos dados

"Rush Hour"→1 = horario de pico (6-9h e 16-19h) | 0 = fora do pico

"Alcohol Involved" →1 = Envolveu álcool | 0 = Não envolveu

"Work Zone" → 1 = Sim | 0 = Não

"Aling" → 1 = Reta | 2 = 0 = curva

"Weekday" → 1 = dia de semana | 0 = fim de semana

"Accident at Intersection → 1 = acidente em cruzamento/interseção | 0 = fora do cruzamento

"Accident at Roadway" → 1 = acidente na via | 0 = fora da via

"Speed Limit" → 

"Number of Vehicle Involved" → 

"Weather" →  1 = "Yes" = chuva, neve ou outra condição adversa | 0 = "No" = sem condições adversas

"Injury" → 1 = houve ferimento | 0 = 2 = não houve

In [0]:
df["Injury"] = df["Injury"].replace({1: 1, 2: 0})
df["Align"] = df["Align"].replace({1: 1, 2: 0})
df["Weather"] = df["Weather"].replace({"Yes": 1, "No": 0})

df.head()

#### Localizar valores ausentes

print(df.isnull().sum()) → Utilizado para verificar valores ausentes nos dados, adicionando o ".sum" no final, ele irar somar cada valor ausente, para imprimir no final a quantidade exata de valores ausentes da tabelas

print(df[].value_counts()) → Utilizado para a contagem da frequência de valores únicos em uma coluna especifica dentro do DataFrame do Pandas

In [0]:
print(df.isnull().sum())

print("\nContagem de Injury:")
print(df["Injury"].value_counts())

Descobrindo a mediana e a média de "Number of Vehical Involved" para alterar possiveis outlier


In [0]:
mediana = st.median(df["Number of Vehical Involved"])
print(mediana)

media = st.mean(df["Number of Vehical Involved"])
print(media)
df.head(4)

Após analisar os dados "Number of Vehical Involved" opter por excluir o valor maximo, pois interfere diretamente a média dos dados, além de ser um caso exclusivo

In [0]:
valor_maximo = df["Number of Vehical Involved"].max()
valor_minimo = df["Number of Vehical Involved"].min()
print(valor_minimo)
print(valor_maximo)



In [0]:
print(df["Number of Vehical Involved"].describe())

In [0]:
print(valor_maximo)

df.drop(df[df["Number of Vehical Involved"] == valor_maximo].index, inplace = True)
df.head(4)
print(df["Number of Vehical Involved"].describe())

In [0]:
# Módulo de tratamento de outliers
class TrataOutlier:

    def __init__(self, df: pd.DataFrame) -> None:
        self.df = df

    def count_outliers(self, Q1, Q3, IQR, columns):
        cut_off = IQR * 1.5
        temp_df = (self.df[columns] < (Q1 - cut_off)) | (self.df[columns] > (Q3 + cut_off))
        return [len(temp_df[temp_df[col] == True]) for col in temp_df]

    def calc_skew(self, columns=None):
        if columns == None:
            columns = self.df.columns
        return [self.df[col].skew() for col in columns]

    def percentage(self, list):
        return [str(round(((value/150001) * 100), 2)) + '%' for value in list]

    def remove_outliers(self, columns):
        for col in columns:
            Q1, Q3 = self.df[col].quantile(0.25), self.df[col].quantile(0.75)
            IQR = Q3 - Q1
            cut_off = IQR * 1.5
            lower, upper = Q1 - cut_off, Q3 + cut_off
            self.df = self.df.drop(self.df[self.df[col] > upper].index)
            self.df = self.df.drop(self.df[self.df[col] < lower].index)

    def replace_outliers_with_fences(self, columns):
        for col in columns:
            Q1, Q3 = self.df[col].quantile(0.25), self.df[col].quantile(0.75)
            IQR = Q3 - Q1
            cut_off = IQR * 1.5
            lower, upper = Q1 - cut_off, Q3 + cut_off

            self.df[col] = np.where(self.df[col] > upper, upper, self.df[col])
            self.df[col] = np.where(self.df[col] < lower, lower, self.df[col])

    def getOverview(self, columns) -> None:
        min = self.df[columns].min()
        Q1 = self.df[columns].quantile(0.25)
        median = self.df[columns].quantile(0.5)
        Q3 = self.df[columns].quantile(0.75)
        max = self.df[columns].max()
        IQR = Q3 - Q1
        skew = self.calc_skew(columns)
        outliers = self.count_outliers(Q1, Q3, IQR, columns)
        cut_off = IQR * 1.5
        lower, upper = Q1 - cut_off, Q3 + cut_off

        new_columns = [
            'Nome de Coluna', 'Min', 'Q1', 'Median', 'Q3', 'Max', 'IQR',
            'Lower fence', 'Upper fence', 'Skew', 'Num_Outliers', 'Percent_Outliers'
        ]
        
        data = zip(
            [column for column in self.df[columns]],
            min, Q1, median, Q3, max, IQR, lower, upper, skew, outliers, self.percentage(outliers)
        )

        new_df = pd.DataFrame(data = data, columns = new_columns)
        new_df.set_index('Nome de Coluna', inplace = True)
        return new_df.sort_values('Num_Outliers', ascending = False).transpose()
      
# Usage example:
# to = TrataOutlier(df)
# display(to.getOverview(["Number of Vehical Involved"]))

In [0]:
housing = fetch_california_housing()
x = housing.data[:, 0:1]
y = housing.target
x_train, x_test, y_train, y_test = tts(
    x,
    y,
    test_size=0.2,
    random_state=42
)
model = LinearRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
print(f"Coeficiente (slope): {model.coef_[0]:.2f}")
print(f"Intercept: {model.intercept_:.2f}")
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse:.2f}")
r2 = r2_score(y_test, y_pred)
print(f"Coeficiente de Determinação (R2): {r2:.2f}")
plt.scatter(x_train, y_train, color="blue", label="Dados de Treino")
plt.plot(x_test, y_pred, color="red", label="Modelo de Regressão Linear")
plt.xlabel("Número de Véiculos")
plt.ylabel("Preço")
plt.title("Teste")
plt.legend()
plt.show()